# Instacart Machine Learning Models
# This notebook builds ML models to predict product reorders.

In [0]:
from pyspark.ml.feature import VectorAssembler, OneHotEncoder, StringIndexer
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.sql.functions import col, count, sum, avg, stddev, min, max


In [0]:
# Prepare training data
train_data = spark.sql("""
  SELECT 
    op.order_id,
    op.product_id,
    op.add_to_cart_order,
    op.reordered,
    o.user_id,
    o.order_dow,
    o.order_hour_of_day,
    o.days_since_prior_order,
    p.aisle_id,
    p.department_id,
    -- Add user-based features
    up.user_total_orders,
    up.user_avg_days_between_orders,
    -- Add product-based features
    pp.product_reorder_rate,
    pp.product_order_count
  FROM instacart.order_products_train op
  JOIN instacart.orders o ON op.order_id = o.order_id
  JOIN instacart.products p ON op.product_id = p.product_id
  -- Join with pre-calculated user stats
  JOIN (
    SELECT 
      user_id,
      COUNT(DISTINCT order_id) as user_total_orders,
      AVG(days_since_prior_order) as user_avg_days_between_orders
    FROM instacart.orders
    GROUP BY user_id
  ) up ON o.user_id = up.user_id
  -- Join with pre-calculated product stats
  JOIN (
    SELECT
      product_id,
      SUM(CAST(reordered as INT)) / COUNT(*) as product_reorder_rate,
      COUNT(*) as product_order_count
    FROM instacart.order_products_prior
    GROUP BY product_id
  ) pp ON op.product_id = pp.product_id
""")


In [0]:
# Explore the dataset
train_data.describe().show()

# Check class imbalance
train_data.groupBy("reordered").count().show()


+-------+------------------+------------------+-----------------+------------------+------------------+-----------------+------------------+----------------------+-----------------+-----------------+------------------+----------------------------+--------------------+-------------------+
summary| order_id| product_id|add_to_cart_order| reordered| user_id| order_dow| order_hour_of_day|days_since_prior_order| aisle_id| department_id| user_total_orders|user_avg_days_between_orders|product_reorder_rate|product_order_count|
+-------+------------------+------------------+-----------------+------------------+------------------+-----------------+------------------+----------------------+-----------------+-----------------+------------------+----------------------------+--------------------+-------------------+
 count| 1384608| 1384608| 1384608| 1384608| 1384608| 1384608| 1384608| 1384608| 1384608| 1384608| 1384608| 1384608| 1384608| 1384608|
 mean|1706294.5879201912| 25556.1871482759| 8.75802032055282|0.5985983036353971|103112.80007771152|2.701386962952691|13.577586580461762| 17.06615663061314|71.30419584460006|9.839725756315145|17.091412154198157| 13.371078257934213| 0.5789612676288105| 34345.384440217014|
 stddev| 989733.3205064402|14121.265456695077|7.423914934289983|0.4901821580348021| 59487.09192379809|2.167645820946835| 4.238455644549526| 10.426420138389979|38.10413801874703| 6.29280868579242|16.614069212739533| 5.415246798393677| 0.1533254128555284| 77656.18902957812|
 min| 1| 1| 1| 0| 1| 0| 0| 0.0| 1| 1| 4| 0.0| 0.0| 1|
 max| 3421070| 49688| 80| 1| 206209| 6| 23| 30.0| 99| 9| 100| 27.0| 0.9411764705882353| 472565|
+-------+------------------+------------------+-----------------+------------------+------------------+-----------------+------------------+----------------------+-----------------+-----------------+------------------+----------------------------+--------------------+-------------------+

+---------+------+
reordered| count|
+---------+------+
 1|828824|
 0|555784|
+---------+------+

In [0]:
# Prepare feature columns
cat_cols = ["order_dow", "aisle_id", "department_id"]
num_cols = ["add_to_cart_order", "order_hour_of_day", "days_since_prior_order", 
            "user_total_orders", "user_avg_days_between_orders",
            "product_reorder_rate", "product_order_count"]

# Create a pipeline for preprocessing
indexers = [StringIndexer(inputCol=col, outputCol=col+"_idx").fit(train_data) for col in cat_cols]
encoders = [OneHotEncoder(inputCol=col+"_idx", outputCol=col+"_enc") for col in cat_cols]

# Get encoded column names
encoded_cols = [col+"_enc" for col in cat_cols]
feature_cols = num_cols + encoded_cols

# Create the assembler
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")


In [0]:
# Split the data
train, test = train_data.randomSplit([0.8, 0.2], seed=42)


In [0]:
# Create and train Random Forest model
rf_pipeline = Pipeline(stages=indexers + encoders + [assembler, 
                                                     RandomForestClassifier(labelCol="reordered", 
                                                                            featuresCol="features",
                                                                            numTrees=100)])
rf_model = rf_pipeline.fit(train)

# Make predictions
rf_predictions = rf_model.transform(test)

# Evaluate model
evaluator = BinaryClassificationEvaluator(labelCol="reordered")
rf_auc = evaluator.evaluate(rf_predictions)
print(f"Random Forest AUC: {rf_auc}")


Random Forest AUC: 0.7196191562661399

In [0]:
# Create and train Logistic Regression model
lr_pipeline = Pipeline(stages=indexers + encoders + [assembler, 
                                                     LogisticRegression(labelCol="reordered", 
                                                                       featuresCol="features")])
lr_model = lr_pipeline.fit(train)

# Make predictions
lr_predictions = lr_model.transform(test)

# Evaluate model
lr_auc = evaluator.evaluate(lr_predictions)
print(f"Logistic Regression AUC: {lr_auc}")


Logistic Regression AUC: 0.7334881438781733

In [0]:
# Save the best model
best_model_path = "/models/product_reorder_prediction"
if rf_auc > lr_auc:
    rf_model.write().overwrite().save(best_model_path)
    print(f"Random Forest model saved to {best_model_path}")
else:
    lr_model.write().overwrite().save(best_model_path)
    print(f"Logistic Regression model saved to {best_model_path}")


Logistic Regression model saved to /models/product_reorder_prediction